# [手把手教你完成一个数据科学小项目（5）：省份提取与可视化]

截至目前我们已经完成了[数据爬取]、[数据提取与IP查询]、[数据异常与清洗]、[评论数变化情况分析]，本文继续对地理信息进行处理，并分别提取出省份和城市数据，从而可以用 pyecharts 进行地图可视化。 

## 读取数据

In [2]:
import pandas as pd
df = pd.read_csv('Sina_Finance_Comments_All_20180811_Cleaned.csv',encoding='utf-8')
df.head(2)

Unnamed: 0  Unnamed: 0.1  No  page             nick                 time  \
0           0            20  21     2            我阿訇打钱  2018-08-11 17:15:13   
1           1            21  22     2  TKK_Questioning  2018-08-11 17:04:20   

        content       area              ip  \
0  [费解][费解][费解]  青海海南藏族自治州  36.100.157.102   
1  那你以为不单身就能避免？       安徽合肥    36.7.153.141   

                                              ip_loc    ...      channel  hot  \
0  青海省海南州 电信 * GeoIP: Hangzhou, Zhejiang, China *...    ...           cj    0   
1  安徽省合肥市 电信 * GeoIP: Hefei, Anhui, China * China...    ...           cj    0   

   level login_type  media_type                                  mid  \
0      5          0           0  5B6EA91F-24649D66-17A3D8EEA-8C5-8D1   
1      4          0           0  5B6EA690-2407998D-1343F7922-8C5-877   

        stamp    time_ymd  time_mdh  cmntcount  
0  1533978913  2018-08-11  08-11 17       3773  
1  1533978260  2018-08-11  08-11 17       3772  

[2 rows x 23 columns]

本文只对`'area'`列进行处理，`'ip_loc'`列读者可自行探索，比如提取并分析下移动、电信、联通等的占比等。

In [3]:
df[['area','ip_loc']]

area                                             ip_loc
0        青海海南藏族自治州  青海省海南州 电信 * GeoIP: Hangzhou, Zhejiang, China *...
1             安徽合肥  安徽省合肥市 电信 * GeoIP: Hefei, Anhui, China * China...
2             山西太原  山西省太原市 电信 * GeoIP: Xian, Shaanxi, China * Chin...
3             广东广州  广东省广州市 移动 * GeoIP: Guangzhou, Guangdong, China...
4             广东深圳  广东省深圳市 电信 * GeoIP: Shenzhen, Guangdong, China ...
5               湖南  湖南省 移动 * GeoIP: Changsha, Hunan, China * China...
6             四川成都  四川省成都市 联通 * GeoIP: Chengdu, Sichuan, China * U...
7               浙江  浙江省 移动 * GeoIP: Hangzhou, Zhejiang, China * Ch...
8             广西南宁  广西自治区河池市 电信 * GeoIP: Nanning, Guangxi, China *...
9             浙江杭州  浙江省杭州市 移动 * GeoIP: Hangzhou, Zhejiang, China *...
10            山西长治  山西省长治市 联通 * GeoIP: Changzhi, Shanxi, China * C...
11              上海  上海市 电信 * GeoIP: Shanghai, China * China Teleco...
12            湖北武汉  湖北省武汉市 电信 * GeoIP: Wuhan, Hubei, China * China...
13            江苏徐州  江苏省徐州市 电信 * GeoIP: Xuzhou, Jiangsu, China * Ch...
14              上海  中国 移动 * GeoIP: Shanghai, China * Shanghai Mobi...
15            江苏泰州   江苏省泰州市 移动 * GeoIP: Beijing, China * China Mobile
16              上海  上海市 翰威科技 * GeoIP: Shanghai, China * China Tele...
17            江苏南京  江苏省南京市 电信 * GeoIP: Nanjing, Jiangsu, China * C...
18            江西南昌  江西省 移动 * GeoIP: Nanchang, Jiangxi, China * Chi...
19            江苏南京  江苏省南京市 电信 * GeoIP: Nanjing, Jiangsu, China * C...
20            湖北黄冈  湖北省黄冈市 电信 * GeoIP: Wuxue, Hubei, China * China...
21            四川成都  四川省成都市 电信 * GeoIP: Chengdu, Sichuan, China * C...
22       青海海南藏族自治州  青海省海南州 电信 * GeoIP: Xining, Qinghai, China * Ch...
23              上海  上海市 电信 * GeoIP: Shanghai, China * China Teleco...
24            辽宁抚顺  辽宁省 电信 * GeoIP: Shenyang, Liaoning, China * Ch...
25            江苏徐州  江苏省 移动 * GeoIP: Yangzhou, Jiangsu, China * Chi...
26              北京  北京市 电信 * GeoIP: Beijing, China * ChinaTelecom ...
27            广东广州  广东省广州市 电信 * GeoIP: Guangzhou, Guangdong, China...
28            浙江绍兴  浙江省杭州市 联通 * GeoIP: Hangzhou, Zhejiang, China *...
29            四川内江  四川省内江市 电信 * GeoIP: Chengdu, Sichuan, China * C...
...            ...                                                ...
3743          山东淄博  山东省淄博市 电信 * GeoIP: Jinan, Shandong, China * Ch...
3744          江苏南京  江苏省南京市 电信 * GeoIP: Nanjing, Jiangsu, China * C...
3745            上海  上海市 电信 * GeoIP: Shanghai, China * China Teleco...
3746            上海  上海市 移动 * GeoIP: Shanghai, China * Shanghai Mob...
3747          四川成都  四川省成都市 电信 * GeoIP: Chengdu, Sichuan, China * C...
3748          广东深圳  广东省深圳市 移动 * GeoIP: Shenzhen, Guangdong, China ...
3749          浙江杭州  浙江省杭州市 电信 * GeoIP: Hangzhou, Zhejiang, China *...
3750          江苏徐州  江苏省南京市 电信 * GeoIP: Wuxi, Jiangsu, China * Chin...
3751          河南安阳  河南省安阳市 联通 * GeoIP: Anyang, Henan, China * Chin...
3752          四川绵阳  四川省绵阳市 电信 * GeoIP: Mianyang, Sichuan, China * ...
3753          河北廊坊  河北省张家口市 移动 * GeoIP: Sanhe, Hebei, China * Hebe...
3754          江苏南京  江苏省南京市 电信 * GeoIP: Nanjing, Jiangsu, China * C...
3755          辽宁沈阳  辽宁省沈阳市 移动 * GeoIP: Shenyang, Liaoning, China *...
3756          安徽合肥  安徽省合肥市 移动 * GeoIP: Hefei, Anhui, China * China...
3757          甘肃兰州  甘肃省 电信 * GeoIP: Lanzhou, Gansu, China * China ...
3758            北京  北京市 联通 * GeoIP: Beijing, China * China Unicom ...
3759            吉林  吉林省吉林市 联通 * GeoIP: Changchun, Jilin, China * C...
3760           新加坡         新加坡  * GeoIP: Singapore * StarHub Internet
3761            北京  北京市 联通 * GeoIP: Beijing, China * China Unicom ...
3762          江西赣州  江西省赣州市 电信 * GeoIP: Ganzhou, Jiangxi, China * C...
3763          河南新乡  河南省新乡市 联通 * GeoIP: Xinxiang, Henan, China * Ch...
3764  湖北恩施土家族苗族自治州  湖北省恩施州 电信 * GeoIP: Wuhan, Hubei, China * China...
3765          福建泉州  福建省泉州市 电信 * GeoIP: Quanzhou, Fujian, China * C...
3766          云南曲靖  云南省昆明市 联通 * GeoIP: Kunming, Yunnan, China * Ch...
3767            上海  上海市 移动 * GeoIP: Shanghai, China * Shanghai Mob...
3768          辽宁大连  辽宁省大

## 统计 area
看看`area`列有多少种情况。

In [4]:
area_count = df.groupby('area')['area'].count().sort_values(ascending=False)
area_name = list(area_count.index)
area_values = area_count.values
print(len(area_name),len(area_values))
print(area_count)

337 337
area
北京              319
上海              281
广东广州            176
四川成都            136
广东深圳            131
湖北武汉            113
重庆               96
江苏南京             96
浙江杭州             87
陕西西安             73
福建福州             73
浙江               68
江苏苏州             64
安徽合肥             52
天津               44
山东济南             44
江苏徐州             42
江苏无锡             42
辽宁沈阳             40
山东青岛             40
江西南昌             39
河南郑州             39
香港               38
广东佛山             38
广东               37
湖南长沙             36
云南昆明             31
北京海淀             30
山西太原             30
广西南宁             29
               ... 
广东梅州              1
菲律宾西米沙鄢           1
菲律宾               1
澳大利亚澳大利亚首都领地      1
广东云浮              1
瑞士                1
美国弗吉尼亚州           1
辽宁铁岭              1
安哥拉               1
四川达州              1
四川遂宁              1
河北邢台              1
湖南怀化              1
陕西商洛              1
美国宾夕法尼亚州          1
意大利皮埃蒙特           1
宁夏固原              1
意大利托斯卡纳           1
意大利    

全部3000多条评论数据里，地理信息`area_name`共337个唯一值，为了分别对省份和城市进行统计和可视化，需要从`area_name`里找出可以分离出省份和城市的提取方法，以便`apply`应用到`area`列上。

**读者可在此处暂停思考下，自己的思路是什么？该如何实现？**

In [5]:
print(area_name)

['北京', '上海', '广东广州', '四川成都', '广东深圳', '湖北武汉', '重庆', '江苏南京', '浙江杭州', '陕西西安', '福建福州', '浙江', '江苏苏州', '安徽合肥', '天津', '山东济南', '江苏徐州', '江苏无锡', '辽宁沈阳', '山东青岛', '江西南昌', '河南郑州', '香港', '广东佛山', '广东', '湖南长沙', '云南昆明', '北京海淀', '山西太原', '广西南宁', '广东东莞', '甘肃兰州', '澳大利亚', '内蒙古呼和浩特', '河南开封', '福建', '辽宁大连', '河北', '河北石家庄', '江苏南通', '黑龙江哈尔滨', '湖南', '浙江绍兴', '云南', '山东', '日本', '吉林', '吉林长春', '辽宁盘锦', '浙江宁波', '福建厦门', '河南', '贵州贵阳', '浙江金华', '贵州', '山西晋城', '浙江温州', '山东临沂', '四川', '江苏常州', '河南洛阳', '新疆乌鲁木齐', '广东汕头', '江苏扬州', '山东淄博', '四川内江', '江苏泰州', '福建泉州', '广东中山', '山东烟台', '英国英格兰', '浙江嘉兴', '内蒙古包头', '广西', '湖北宜昌', '浙江湖州', '山东济宁', '河北保定', '海南海口', '广东惠州', '河北廊坊', '江苏连云港', '新加坡', '辽宁', '山东潍坊', '安徽淮北', '山西大同', '广西柳州', '湖北襄阳', '浙江台州', '四川绵阳', '河北邯郸', '江西九江', '河南周口', '安徽芜湖', '浙江丽水', '美国', '宁夏银川', '河南南阳', '河北承德', '河北唐山', '江苏盐城', '陕西宝鸡', '青海西宁', '广东揭阳', '河北秦皇岛', '天津塘沽', '河北沧州', '山西长治', '广东韶关', '湖北', '江西上饶', '广西桂林', '江西宜春', '辽宁朝阳', '黑龙江', '湖南常德', '辽宁营口', '湖北黄冈', '辽宁鞍山', '贵州遵义', '山东聊城', '山西', '河南安阳', '安徽六安', '山西运城', '山东德州', '山东东营', '河北衡水', '

### 数据处理思路
首先再次明确下这次的目的是提取出省份和城市信息，且由于数据量不大，所以后续只在中国地图上进行可视化，因而海外地理信息统一可以筛选出去，实现的方式是按照国家名手动构建一个`unchina`的列表，用来存储本次数据里出现的海外国家，然后遍历所有的337条`area_name`元素，包含这些国家名的就添加到`drop`列表里，然后根据其他国内的地理信息的长度分别打印出来，这样数据就清晰多了！过程中可能有些会被误分，需要核查一遍。

In [6]:
area_len_2 = []
area_len_3 = []
area_len_4 = []
area_len_5 = []
unchina = ['英国','美国','日本','瑞士','法国','瑞典','越南','泰国',
           '意大利','加拿大','菲律宾','新加坡','新西兰','伊拉克','爱尔兰','安哥拉',
           '澳大利亚', '大韩民国', '马来西亚']
droped = []
for area in area_name:
    for unarea in unchina:
        if unarea in area: 
            droped.append(area)
    if len(area)==2 and area not in droped: area_len_2.append(area) # 我国共有34个省级行政区域，包括23个省，5个自治区，4个直辖市，2个特别行政区。
    if len(area)==3 and area not in droped: area_len_3.append(area)
    if len(area)==4 and area not in droped: area_len_4.append(area)
    if len(area)>=5 and area not in droped: area_len_5.append(area)
print(len(droped),'\n', droped)
print(len(area_len_2),'\n', area_len_2)
print(len(area_len_3),'\n', area_len_3)
print(len(area_len_4),'\n', area_len_4)
print(len(area_len_5),'\n', area_len_5)

47 
 ['澳大利亚', '日本', '英国英格兰', '新加坡', '美国', '美国加利福尼亚州', '美国纽约州', '美国伊利诺伊州', '大韩民国', '英国', '法国', '美国马里兰州', '加拿大安大略', '加拿大不列颠哥伦比亚', '泰国', '英国苏格兰', '美国康乃狄克州', '美国德克萨斯州', '美国俄亥俄州', '美国佛罗里达州', '瑞典', '美国俄勒冈州', '马来西亚', '新西兰', '日本和歌山县', '加拿大', '加拿大艾伯塔', '日本岐阜县', '伊拉克', '美国乔治亚', '美国路易斯安那州', '爱尔兰', '美国新泽西州', '越南胡志明市', '菲律宾西米沙鄢', '菲律宾', '澳大利亚澳大利亚首都领地', '瑞士', '美国弗吉尼亚州', '安哥拉', '美国宾夕法尼亚州', '意大利皮埃蒙特', '意大利托斯卡纳', '意大利', '美国密苏里州', '美国密歇根州', '美国田纳西州']
28 
 ['北京', '上海', '重庆', '浙江', '天津', '香港', '广东', '福建', '河北', '湖南', '云南', '山东', '吉林', '河南', '贵州', '四川', '广西', '辽宁', '湖北', '山西', '江苏', '宁夏', '新疆', '甘肃', '江西', '澳门', '青海', '海南']
2 
 ['黑龙江', '内蒙古']
223 
 ['广东广州', '四川成都', '广东深圳', '湖北武汉', '江苏南京', '浙江杭州', '陕西西安', '福建福州', '江苏苏州', '安徽合肥', '山东济南', '江苏徐州', '江苏无锡', '辽宁沈阳', '山东青岛', '江西南昌', '河南郑州', '广东佛山', '湖南长沙', '云南昆明', '北京海淀', '山西太原', '广西南宁', '广东东莞', '甘肃兰州', '河南开封', '辽宁大连', '江苏南通', '浙江绍兴', '吉林长春', '辽宁盘锦', '浙江宁波', '福建厦门', '贵州贵阳', '浙江金华', '山西晋城', '浙江温州', '山东临沂', '江苏常州', '河南洛阳', '广东汕头', '江苏扬州', '山东淄博', '四川内江', '江苏泰州', '福建

上一步骤按照地理信息是否为国内以及文字长度，对数据进行了划分，有了更清晰的了解后，才能避免提取省份和城市时遇到什么抓瞎的情况。

## 省份汇总
根据百度百科词条里的内容：[省份 - 百科](https://baike.baidu.com/item/%E7%9C%81%E4%BB%BD/1635191)：`我国共有34个省级行政区域，包括23个省，5个自治区，4个直辖市，2个特别行政区。`

复制过来所有省份，先手动去掉自治区和行政区的后缀文字，再用代码去掉无关的文字与字符。

In [7]:
prolist = '北京市，天津市，上海市，重庆市，河北省，山西省，辽宁省，吉林省，江苏省，浙江省，安徽省，福建省，\
江西省，山东省，河南省，湖北省，湖南省，广东省，海南省，四川省，贵州省，云南省，陕西省，甘肃省，\
青海省，台湾省，广西，西藏，宁夏，新疆，香港，澳门，内蒙古，黑龙江省'
prolist = prolist.replace('市', '').replace('省', '').split('，')
print(len(prolist), prolist)

34 ['北京', '天津', '上海', '重庆', '河北', '山西', '辽宁', '吉林', '江苏', '浙江', '安徽', '福建', '江西', '山东', '河南', '湖北', '湖南', '广东', '海南', '四川', '贵州', '云南', '陕西', '甘肃', '青海', '台湾', '广西', '西藏', '宁夏', '新疆', '香港', '澳门', '内蒙古', '黑龙江']


## 提取省份
从`area`列提取出相应省份，非国内的则统一用`海外`表示

In [8]:
def get_pro(area):
    prolist = ['北京', '天津', '上海', '重庆', '河北', '山西', '辽宁', '吉林', '江苏', '浙江', '安徽', '福建', '江西', '山东', '河南', '湖北', '湖南', '广东', '海南', '四川', '贵州', '云南', '陕西', '甘肃', '青海', '台湾', '广西', '西藏', '宁夏', '新疆', '香港', '澳门', '内蒙古', '黑龙江']
    for pro in prolist:
        if pro in area:
            return pro
    return "海外"
df['pro'] = df.area.apply(get_pro)
df[['area','pro']]

area pro
0        青海海南藏族自治州  海南
1             安徽合肥  安徽
2             山西太原  山西
3             广东广州  广东
4             广东深圳  广东
5               湖南  湖南
6             四川成都  四川
7               浙江  浙江
8             广西南宁  广西
9             浙江杭州  浙江
10            山西长治  山西
11              上海  上海
12            湖北武汉  湖北
13            江苏徐州  江苏
14              上海  上海
15            江苏泰州  江苏
16              上海  上海
17            江苏南京  江苏
18            江西南昌  江西
19            江苏南京  江苏
20            湖北黄冈  湖北
21            四川成都  四川
22       青海海南藏族自治州  海南
23              上海  上海
24            辽宁抚顺  辽宁
25            江苏徐州  江苏
26              北京  北京
27            广东广州  广东
28            浙江绍兴  浙江
29            四川内江  四川
...            ...  ..
3743          山东淄博  山东
3744          江苏南京  江苏
3745            上海  上海
3746            上海  上海
3747          四川成都  四川
3748          广东深圳  广东
3749          浙江杭州  浙江
3750          江苏徐州  江苏
3751          河南安阳  河南
3752          四川绵阳  四川
3753          河北廊坊  河北
3754          江苏南京  江苏
3755          辽宁沈阳  辽宁
3756          安徽合肥  安徽
3757          甘肃兰州  甘肃
3758            北京  北京
3759            吉林  吉林
3760           新加坡  海外
3761            北京  北京
3762          江西赣州  江西
3763          河南新乡  河南
3764  湖北恩施土家族苗族自治州  湖北
3765          福建泉州  福建
3766          云南曲靖  云南
3767            上海  上海
3768          辽宁大连  辽宁
3769          湖南长沙  湖南
3770           伊拉克  海外
3771          广东佛山  广东
3772          广东深圳  广东

[3773 rows x 2 columns]

## 统计数据

In [9]:
pro_count = df.groupby('pro')['pro'].count().sort_values(ascending=False)
pro_count

pro
广东     472
北京     352
江苏     323
上海     284
浙江     249
四川     191
山东     175
湖北     153
福建     135
海外     133
河南     127
辽宁     115
河北      96
重庆      96
陕西      92
安徽      88
湖南      86
山西      74
江西      68
广西      67
云南      64
内蒙古     51
天津      49
香港      38
甘肃      36
黑龙江     36
贵州      32
吉林      31
新疆      21
海南      16
宁夏      12
青海       8
澳门       2
西藏       1
Name: pro, dtype: int64

## pyecharts
[pyecharts 自定义主题](http://pyecharts.org/#/zh-cn/themes)   

先安装主题插件  
`pip install echarts-themes-pypkg`
### 省份分布柱形图

In [10]:
from pyecharts import Bar
bar = Bar("省份分布")
bar.use_theme("macarons") # 换主题
bar.add("省份", pro_count.index, pro_count.values,is_label_show=True,xaxis_interval=0,xaxis_rotate=-45)
bar

### 省份分布地图

In [11]:
from pyecharts import Map
mapp = Map("省份分布情况", width=1000, height=600)
#mapp.use_theme("macarons") # 换主题
mapp.add("", pro_count.index, pro_count.values, maptype='china', is_visualmap=True,
         visual_range=[0, 480], is_map_symbol_show=False, visual_text_color='#000', is_label_show=True)
mapp